In [1]:
import os

# Dask Configuration

In [2]:
from dask_jobqueue import PBSCluster
from pathlib import Path

# Define the working directory path
working_directory = str(Path.cwd())

# Launch a scheduler and workers on HPC via PBS
cluster = PBSCluster(
     cores=4,
     memory="8GB",
     processes=1,
     queue="tamirQ",
     walltime="02:30:00",
     scheduler_options={"dashboard_address": ":12435"},  # FIXME - this is the port we need to define...
     # Additional custom options
     log_directory="dask-logs",
     #worker_extra_args=["--lifetime", "25m", "--lifetime-stagger", "4m"],  # for walltime="00:30:00"
     job_script_prologue=[f"cd {working_directory}"]
)

In [3]:
cluster

Dashboard: http://132.66.112.146:12435/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://132.66.112.146:38052,Workers: 0
Dashboard: http://132.66.112.146:12435/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
cluster.adapt(minimum=30, maximum=60)
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -q tamirQ
#PBS -l select=1:ncpus=4:mem=7630MB
#PBS -l walltime=02:30:00
#PBS -e dask-logs/
#PBS -o dask-logs/
cd /tamir2/moranb/microbiome/Igem_TAU_2021
/tamir2/moranb/microbiome/Igem_TAU_2021/venv/bin/python -m distributed.cli.dask_worker tcp://132.66.112.146:38052 --nthreads 4 --memory-limit 7.45GiB --name dummy-name --nanny --death-timeout 60



In [5]:
from dask.distributed import Client, progress, wait, get_client, get_worker
client = Client(cluster)

In [6]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://132.66.112.146:12435/status,
Dashboard: http://132.66.112.146:12435/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://132.66.112.146:38052,Workers: 0
Dashboard: http://132.66.112.146:12435/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [7]:
import dask.bag as db
import dask.dataframe as dd
from dask import delayed, compute, persist
import json
import matplotlib
import numpy as np
import pandas as pd

# Analysis for endogenous genes

In [8]:
from Bio import SeqIO
from analysis.orf_model_analysis.input_testing_data.generate_input_testing_data_for_modules import generate_testing_data
from analysis.orf_model_analysis.input_testing_data.generate_input_testing_data_for_modules import generate_testing_data_for_ecoli_and_bacillus

from modules.main import run_modules

In [9]:
output_path = "/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/endogenous_genes"

In [10]:
def convert_to_json_result(x):
    gene_name = x[0]
    result = run_modules(x[1], should_run_output_module=False)
    return {
        "initial_optimization_score": result.get("initial_optimization_score"),
        "final_optimization_score": result.get("final_optimization_score"),
        "average_distance_score": result.get("average_distance_score"),
        "weakest_link_score": result.get("weakest_link_score"),
        "gene_name": gene_name,
    }

In [11]:
def test_partial(x):
    gene_name = x[0]
    result = run_modules(x[1], should_run_output_module=False)
    return result

## Paenibacillus-prosopidis

In [12]:
# organism = "Paenibacillus-aceris"
organism = "Terrabacter-tumescens"
fasta_file_path = f"/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/example_data/arabidopsis_microbiome/{organism}.fasta"
with open(fasta_file_path, "r") as fasta_handle:
    genome_dict = SeqIO.to_dict(SeqIO.parse(fasta_handle, "fasta"))


### Two wanted hosts and two unwanted hosts

In [13]:
optimization_cub_index = "CAI"
# optimization_method = "zscore_bulk_aa_diff"
optimization_method = "single_codon_weakest_link"

wanted_hosts = [
    "Paenibacillus-aceris.gb", 
    "Arthrobacter-luteolus.gb",
]
unwanted_hosts = [
    "Terrabacter-tumescens.gb", 
    "Rhodanobacter-denitrificans.gb",
]

configuration = f"wanted_{'_'.join([host[:-3] for host in wanted_hosts])}_unwanted_{'_'.join([host[:-3] for host in unwanted_hosts])}"
configuration_output_path = os.path.join(output_path, configuration, organism)

In [14]:
inputs = [(gene_name, generate_testing_data(
    optimization_method=optimization_method,
    optimization_cub_index=optimization_cub_index,
    wanted_hosts=wanted_hosts,
    unwanted_hosts=unwanted_hosts,
    tuning_param=0.5,
    sequence=str(orf_sequence.seq),
    output_path=os.path.join(configuration_output_path,gene_name))) for gene_name, orf_sequence in genome_dict.items()]

In [15]:
inputs_series = pd.Series(inputs)
inputs_db = db.from_sequence(inputs)

An example for running the code on some inputs and saving the results to a file:

In [16]:
# %%time
# partial_inputs = inputs_db.take(10)
# results_db = db.from_sequence(partial_inputs).map(lambda x: {x[0]: run_modules(x[1], should_run_output_module=False)})
# results_db.map(json.dumps).to_textfiles(os.path.join(configuration_output_path, 'results/*.json')) 

An example for reading and plotting the results:

In [17]:
# df = dd.read_json([F"{configuration_output_path}/results/{i}.json" for i in range(3)], orient='index')
# df[["weakest_link_score", "average_distance_score"]].compute().hist()

In [18]:
# %%time
# results_db = inputs_db.map(convert_to_json_result)
# results_db.map(json.dumps).to_textfiles(os.path.join(configuration_output_path, 'results/*.json')) 

In [19]:
# results_dir = F"{configuration_output_path}/zscore_bulk_aa_diff/"
# df = dd.read_json([F"{results_dir}/{file}" for file in os.listdir(results_dir)], orient='index')
# df[["weakest_link_score", "average_distance_score"]].compute().hist()

## Bacillus and E.coli

In [20]:
# organism = "Escherichia-coli"
# organism = "Bacillus-subtilis"
# fasta_file_path = f"/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/example_data/{organism}.fasta"
# with open(fasta_file_path, "r") as fasta_handle:
#     genome_dict = SeqIO.to_dict(SeqIO.parse(fasta_handle, "fasta"))

# optimization_cub_index = "CAI"
# optimization_method = "single_codon_diff"
# is_ecoli_optimized = True

# configuration = f"e_coli_optimized_{is_ecoli_optimized}_bacillus_optimized_{not is_ecoli_optimized}"
# configuration_output_path = os.path.join(output_path, configuration, organism)

# inputs = [(gene_name, generate_testing_data_for_ecoli_and_bacillus(
#     optimization_method=optimization_method,
#     optimization_cub_index=optimization_cub_index,
#     is_ecoli_optimized=is_ecoli_optimized,
#     tuning_param=0.5,
#     sequence=str(orf_sequence.seq),
#     output_path=os.path.join(configuration_output_path,gene_name))) for gene_name, orf_sequence in genome_dict.items()]

# inputs_series = pd.Series(inputs)
# inputs_db = db.from_sequence(inputs)

In [21]:
%%time
results_db = inputs_db.map(convert_to_json_result)
results_db.map(json.dumps).to_textfiles(os.path.join(configuration_output_path, 'results/*.json'))

# For testing on partial inputs set

# partial_inputs = inputs_db.take(3)
# results_db = db.from_sequence(partial_inputs).map(convert_to_json_result).compute()

CPU times: user 2.13 s, sys: 363 ms, total: 2.49 s
Wall time: 7.27 s


['/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/endogenous_genes/wanted_Paenibacillus-aceris_Arthrobacter-luteolus_unwanted_Terrabacter-tumescens_Rhodanobacter-denitrificans/Terrabacter-tumescens/results/000.json',
 '/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/endogenous_genes/wanted_Paenibacillus-aceris_Arthrobacter-luteolus_unwanted_Terrabacter-tumescens_Rhodanobacter-denitrificans/Terrabacter-tumescens/results/001.json',
 '/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/endogenous_genes/wanted_Paenibacillus-aceris_Arthrobacter-luteolus_unwanted_Terrabacter-tumescens_Rhodanobacter-denitrificans/Terrabacter-tumescens/results/002.json',
 '/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/endogenous_genes/wanted_Paenibacillus-aceris_Arthrobacter-luteolus_unwanted_Terrabacter-tumescens_Rhodanobacter-denitrificans/Terrabacter-tumescens/results/003.json',
 '/tamir2/moranb/microbiome/Igem_TAU_2021/analysis/results/endogenous_genes/wanted_Paenibacillus

In [22]:
# %%time
# results_db = inputs_db.map(lambda x: {x[0]: run_modules(x[1], should_run_output_module=False)})
# results_db.map(json.dumps).to_textfiles(os.path.join(configuration_output_path, 'results/*.json')) 

In [23]:
results_dir = F"{configuration_output_path}/{optimization_method}/"
os.rename(F"{configuration_output_path}/results", results_dir)

# df = dd.read_json([F"{results_dir}/{file}" for file in os.listdir(results_dir)], orient='records', lines=True)

### Weakest Link Score

In [24]:
# df["weakest_link_score"].compute().hist()

### Average Distance Score

In [25]:
# df["average_distance_score"].compute().hist()